In [1]:
import csv
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np


def get_feature_labels(file_name):
    with open(file_name, 'r+') as f:
        reader = csv.reader(f)
        next(reader, None)
        features = []
        labels = []
        ignore = ['#N/A', 7, 9]
        lengths = set()
        for row in reader:
            if row[-1] in ignore:
                continue
            str_features = row[1:-1]
            l = row[-1]
            feature_entry = []
            for feat in str_features:
                try:
                    feature_entry.append(float(feat))
                except:
                    feature_entry.append(-1.0)
            l = int(l)
            lengths.add(len(feature_entry))
            labels.append(l-1)
            features.append(feature_entry)
    return features, labels


f, l = [], []

for data in ['./Dataset_v3.csv']:
    feat, lab = get_feature_labels(data)
    f += feat
    l += lab

f = np.array(f)
l = np.array(l)

feat_train, feat_test, label_train, label_test = train_test_split(f, l,
                                              test_size=0.5, random_state=42)
model = Sequential()
model.add(Dense(64, input_dim=9, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop',
                   metrics=['accuracy', 'precision', 'recall'])

model.fit(feat_train, label_train, nb_epoch=10, batch_size=32, verbose=False)

print(model.evaluate(feat_test, label_test, batch_size=32, verbose=False))

# to supress common keras error
import gc
gc.collect()


Using TensorFlow backend.


[0.12259915436652505, 0.94069170491239951, 0.94906070041708035, 0.99928598048332029]


0